In [54]:
import numpy as np
import pandas as pd
import math
from statsmodels.tsa.ar_model import AR

In [3]:
df = pd.read_excel("C:/Users/kz2329/Desktop/ts/BestPair_Distance Method_and_Cointegration Method.xlsx")

In [4]:
df.head()

,Date,MU,SWKS,MU_SWKS
0,2000-01-03,37.97,29.58,19.541875
1,2000-01-04,36.50,27.84,19.155883
2,2000-01-05,36.38,28.56,18.587328
3,2000-01-06,37.00,25.78,20.939248
4,2000-01-07,35.25,28.43,17.538317


In [7]:
df.tail()

,Date,MU,SWKS,MU_SWKS
4758,2018-11-29,37.91,71.23,-6.465771
4759,2018-11-30,38.56,72.77,-6.775180
4760,2018-12-03,40.03,73.46,-5.735045
4761,2018-12-04,36.88,70.76,-7.202965
4762,2018-12-06,37.67,70.61,-6.319516


In [6]:
df['Date'] = pd.to_datetime(df['Date'])

In [20]:
ins = df.loc[(df['Date'] >= pd.datetime(2010,1,1)) & (df['Date'] < pd.datetime(2017,1,1))]
outs = df.loc[(df['Date'] >= pd.datetime(2017,1,1)) & (df['Date'] < pd.datetime(2018,10,1))]
y = ins['MU_SWKS'].values

In [76]:
def KalmanFilter(A, B, C2, D2, x0, R0, y1):
    xm = A + B*x0
    Rm = B**2 * R0 + C2
    K = Rm / (Rm + D2)
    x1 = xm + K * (y1 - xm)
    R1 = D2 * K
    return x1, R1, Rm, K

In [81]:
def param_update(A, B, C2, D2):
    # Compute Smoothers, this is done by iterating conversely
    n = y.shape[0] - 5    # Leave some backup space
    x_kn = np.zeros(n+1)
    sig_kn = np.zeros(n+1)
    sig_k1kn = np.zeros(n+1)
    x_kk = np.zeros(n+2)
    sig_kk = np.zeros(n+2)
    sig_kk_1 = np.zeros(n+2)
    K = np.zeros(n+2)

    xi = y[0]
    Ri = D2

    # Compute Kalman Filter values up until n+1, so we will have smoothers from 0 to n
    for i in range(n+2):
        if i == 0:
            x_kk[i] = xi
            sig_kk[i] = Ri
        else:
            xi, Ri, Rm, Ki = KalmanFilter(A, B, C2, D2, xi, Ri, y[i])
            x_kk[i] = xi
            sig_kk[i] = Ri
            sig_kk_1[i] = Rm
            K[i] = Ki

    J = B*sig_kk[:n+1] / sig_kk_1[1:]
    xkn = x_kk[n]
    sigkn = sig_kk[n]
    for i in range(n, -1, -1):
        if i == n:
            x_kn[i] = xkn
            sig_kn[i] = sigkn
            sig_k1kn[i] = B * (1 - K[n]) * sig_kk[n-1]
        elif i == 0:
            x_kn[i] = x_kk[i] + J[i] * (x_kn[i + 1] - (A + B * x_kk[i]))
            sig_kn[i] = sig_kk[i] + J[i]**2 * (sig_kn[i+1] - sig_kk_1[i+1])
        else:
            x_kn[i] = x_kk[i] + J[i] * (x_kn[i + 1] - (A + B * x_kk[i]))
            sig_kn[i] = sig_kk[i] + J[i]**2 * (sig_kn[i+1] - sig_kk_1[i+1])
            sig_k1kn[i] = J[i-1] * sig_kk[i] + J[i] * J[i-1] * (sig_k1kn[i+1] - B * sig_kk[i])
            
    alpha = np.sum(sig_kn[:n] + np.square(x_kn[:n]))
    beta = np.sum(sig_k1kn[1:] + x_kn[:n] * x_kn[1:])
    gamma = np.sum(x_kn[1:])
    delta = gamma - x_kk[n] + x_kn[0]
    
    An = (alpha * gamma - delta * beta) / (n * alpha - delta**2)
    Bn = (n * beta - gamma * delta) / (n * alpha - delta**2)
    Cn2 = np.sum(sig_kn[1:] + np.square(x_kn[1:]) + An**2 + Bn**2 * sig_kn[:n] + Bn**2 * np.square(x_kn[:n])
                - 2 * An * x_kn[1:] + 2 * An * Bn * x_kn[:n] - 2 * Bn * sig_k1kn[1:] - 2 * Bn * x_kn[1:] * x_kn[:n]) / n
    Dn2 = np.sum(np.square(y[:n+1]) - 2 * y[:n+1] * x_kn + sig_kn + np.square(x_kn)) / (n + 1)
    return An, Bn, Cn2, Dn2

In [ ]:
mu = np.mean(y)
sig = np.std(y)
B = 0.8
A = mu * (1 - B)
C2 = sig**2 * (1 - B**2)
D2 = C2
i = 1
while(True):
    if i % 1000 == 1:
        print("Iteration: %d: " % (int(i/1000) + 1))
    An, Bn, Cn2, Dn2 = param_update(A, B, C2, D2)
    if abs((An - A) / A) < 0.1 and abs((Bn - B) / B) < 0.1 and abs((Cn2 - C2) / C2) < 0.1 and abs((Dn2 - D2) / D2) < 0.1:
        break
    i += 1

Iteration: 1: 
Iteration: 2: 
Iteration: 3: 
Iteration: 4: 
Iteration: 5: 
Iteration: 6: 
Iteration: 7: 
Iteration: 8: 
Iteration: 9: 
Iteration: 10: 
Iteration: 11: 
Iteration: 12: 
Iteration: 13: 
Iteration: 14: 
Iteration: 15: 
Iteration: 16: 
Iteration: 17: 
Iteration: 18: 
Iteration: 19: 
Iteration: 20: 
Iteration: 21: 
Iteration: 22: 
Iteration: 23: 
Iteration: 24: 
Iteration: 25: 
Iteration: 26: 
Iteration: 27: 
Iteration: 28: 
Iteration: 29: 
Iteration: 30: 
Iteration: 31: 
Iteration: 32: 
Iteration: 33: 
Iteration: 34: 
Iteration: 35: 
Iteration: 36: 
Iteration: 37: 
Iteration: 38: 
Iteration: 39: 
Iteration: 40: 
Iteration: 41: 
Iteration: 42: 
Iteration: 43: 
Iteration: 44: 
Iteration: 45: 
Iteration: 46: 
Iteration: 47: 
Iteration: 48: 
Iteration: 49: 
Iteration: 50: 
Iteration: 51: 
Iteration: 52: 
Iteration: 53: 
Iteration: 54: 
Iteration: 55: 
Iteration: 56: 
Iteration: 57: 
Iteration: 58: 
Iteration: 59: 
Iteration: 60: 
Iteration: 61: 
Iteration: 62: 
Iteration: 63: 
I

In [85]:
print(An, Bn, Cn2, Dn2)

-0.0898723652039 0.992994194878 2.34123233394 0.973948241913


In [86]:
print(A, B, C2, D2)

-2.14769511594 0.8 59.956295334 1


In [67]:
s**2

166.5452648167807